In [11]:
#coding=utf-8
import xlwings as xw
import re
from modules.my_libs import han_ji_chu_im_python as ji

#==========================================================
# Excel 檔案
#==========================================================
# file_path = 'to-tek-keng.xlsx'
file_path = 'hoo-goa-chu-im.xlsx'
wb = xw.Book(file_path)

source_sheet = wb.sheets['漢字注音表']
end_row = source_sheet.range('A' + str(source_sheet.cells.last_cell.row)).end('up').row
print(f'end_row = {end_row}')

#==========================================================
# 備妥程式需使用之工作表
#==========================================================
sheet_name_list = ["漢字注音表", "方音符號注音"]
#-----------------------------------------------------
# 檢查工作表是否已存在
for sheet_name in sheet_name_list:
    sheet = wb.sheets[sheet_name]
    try:
        sheet.select()
        continue
    except: 
        # CommandError 的 Exception 發生日，表工作表不存在
        # 新增程式需使用之工作表
        wb.sheets.add(name=sheet_name)

chu_im_sheet = wb.sheets["方音符號注音"]

end_row = 516


In [12]:
#==========================================================
# 自【漢字注音表】讀取：漢字、注音碼；於【方音符號注音】工作表填 
# HTML Ruby Tag 格式的注音
#==========================================================

i = 1       # index for target sheet
html_str = "<div class='zhu_yin'><p>"
# html_str = """
# <div class='zhu_yin'>
#     <p>
# """
chu_im_sheet.range('A' + str(i)).value = html_str
i += 1

row = 1     # index for source sheet
end_counter = end_row + 1

while row < end_counter:
    # 自 source_sheet 取待注音漢字
    han_ji = str(source_sheet.range('A' + str(row)).value)
    han_ji.strip()
    
    #=========================================================
    # 如是空白或換行，處理換行
    #=========================================================
    if han_ji == '' or han_ji == '\n':
        html_str = "</p><p>"
        # html_str = """
        #     </p>
        #     <p>
        # """
        chu_im_sheet.range('A' + str(i)).value = html_str
        i += 1
        row += 1
        continue
    
    #=========================================================
    # 如只是標點符號，不必處理為漢字注音的工作
    #=========================================================
    piau_tiam = r"[；：？！\uFF0C\uFF08-\uFF09\u2013-\u2014\u2026\\u2018-\u201D\u3000\u3001-\u303F]"
    searchObj = re.search(piau_tiam, han_ji, re.M|re.I)
    if searchObj:
        chu_im_sheet.range('A' + str(i)).value = han_ji
        i += 1
        row += 1
        continue
        
    #=========================================================
    # 在字庫中查不到注音的漢字，略過注音處理
    #=========================================================
    chu_im_code = str(source_sheet.range('B' + str(row)).value).strip()
    if chu_im_code == 'None':
        ruby_tag = f'<ruby><rb>{han_ji}</rb><rp>(</rp><rtc></rtc><rp>)</rp></ruby>'
        chu_im_sheet.range('A' + str(i)).value = ruby_tag
        i += 1
        row += 1
        continue
    
    #=========================================================
    # 備妥注音時需參考用的資料
    #=========================================================
    siann_index = int(source_sheet.range('F' + str(row)).value) - 1
    un_index = int(source_sheet.range('G' + str(row)).value) - 1
    tiau_ho =  int(source_sheet.range('E' + str(row)).value)

    #=========================================================
    # 使用注音碼，取得各式之〖 注音／拼音〗
    #=========================================================

    # 輸出十五音
    # sip_ngoo_im_chu_im = ji.get_sip_ngoo_im_chu_im(siann_index, un_index, tiau_ho)

    # 輸出方音符號
    TPS_chu_im = ji.get_TPS_chu_im(siann_index, un_index, tiau_ho)

    # 輸出白話字拼音
    # POJ_chu_im = ji.get_POJ_chu_im(siann_index, un_index, tiau_ho)

    # 輸出羅馬拼音
    # TL_chu_im = ji.get_TL_chu_im(siann_index, un_index, tiau_ho)

    # 輸出閩拼拼音
    # BP_chu_im  = ji.get_BP_chu_im(siann_index, un_index, tiau_ho)
    # BP_chu_im1 = ji.get_BP_chu_im(siann_index, un_index, tiau_ho)

    #=========================================================
    # 將已注音之漢字加入【漢字注音表】
    #=========================================================
    #ruby_tag = f'<ruby><rb>{han_ji}</rb><rp>(</rp><rt>{TPS_chu_im}</rt><rp>)</rp></ruby>'
    ruby_tag = f'  <ruby><rb>{han_ji}</rb><rp>(</rp><rtc>{TPS_chu_im}</rtc><rp>)</rp></ruby>'
    chu_im_sheet.range('A' + str(i)).value = ruby_tag
    
    #=========================================================
    # 調整讀取來源；寫入標的各手標
    #=========================================================
    i += 1
    row += 1

html_str = "</p></div>"
# html_str = """
#   </p>
# </div>
# """
chu_im_sheet.range('A' + str(i)).value = html_str